In [1]:
import pandas as pd
import networkx as nx

In [3]:
thread_char = pd.read_csv('data/thread_character2000.csv',encoding='utf-8-sig')
thread_char['thread_id'] = thread_char['thread_id'].astype('int64').astype(str)
user_char = pd.read_csv('data/user_character400.csv',encoding='utf-8-sig')

In [ ]:
# G_user = nx.Graph()
# G_user.add_nodes_from(range(len(user_char['portrait'])),type = 'user')
# F1 = open("data/user_similarity_400.csv",'r',encoding='utf-8')
# while 1:
#     line = F1.readline()
#     i,j,w = line.split(',')
#     try:
#         G_user.add_edge(int(i),int(j),weight = w)
#     except:
#         print(i,j,w)
#         pass
#     if not line:
#         break
# F1.close()

In [4]:
def construct_network(type,data):
    G_old = nx.from_pandas_edgelist(
        data,
        edge_attr=["weight"],
        create_using=nx.MultiDiGraph,
    )
    for i in range(len(G_old.nodes)):
        G_old.nodes[i]["type"] = type

    data_new = pd.DataFrame()
    data_new['source'] = data['target']
    data_new['target'] = data['source']
    data_new['weight'] = data['weight']
    G_new = nx.from_pandas_edgelist(
        data_new,
        edge_attr=["weight"],
        create_using=nx.MultiDiGraph,
    )
    for i in range(len(G_new.nodes)):
        G_new.nodes[i]["type"] = type
    G = nx.compose(G_old,G_new)
    return G

In [5]:
def normalize(max,min,x):
    y = (x-min)/(max-min)
    return y

In [6]:
users = pd.read_csv('data/user_similarity400_0.6.txt')
standard_w = []
users_max = users['weight'].max()
users_min = users['weight'].min()
for w in users['weight']:
    standard_w.append(normalize(users_max,users_min,w))
users['weight'] = standard_w
users['weight'].mean()

0.47266024048701355

In [7]:
G_user = construct_network('user', users)
del users

In [8]:
threads = pd.read_csv('data/thread_similarity2000_0.4.txt')
standard_w = []
threads_max = threads['weight'].max()
threads_min = threads['weight'].min()
for w in threads['weight']:
    standard_w.append(normalize(threads_max,threads_min,w))
threads['weight'] = standard_w
threads['weight'].mean()

0.34741967583451117

In [9]:
G_thread = construct_network('thread', threads)
del threads

In [10]:
u_t = pd.read_csv('data/user_thread_character400_2000_180_60.csv',encoding='utf-8')
user_thread = pd.DataFrame()
user_thread['source'] = u_t['portrait']
user_thread['target'] = u_t['thread_id'].astype('int64').astype(str)
user_thread['weight'] = u_t['weight']
user_thread['weight'].mean()

0.1730248406743711

In [11]:
G_ut = nx.from_pandas_edgelist(
        user_thread,
        edge_attr=["weight"],
        create_using=nx.MultiDiGraph,
    )
del u_t

In [12]:
user_mapping={}
for i in range(len(user_char['portrait'])):
    user_mapping[i] = user_char['portrait'][i]

thread_mapping={}
for i in range(len(thread_char['thread_id'])):
    thread_mapping[i] = str(thread_char['thread_id'][i])

In [13]:
G_user = nx.relabel_nodes(G_user, user_mapping)
G_thread = nx.relabel_nodes(G_thread, thread_mapping)

In [14]:
G_tem = nx.compose(G_user,G_thread)
G = nx.compose(G_ut,G_tem)

In [15]:
adj = {}
for n, nbrdict in G.adjacency():
    n_e = {}
    for node,d in nbrdict.items():
        n_e[node] = d[0]['weight']
    res = {k: v for k, v in n_e.items()if v > 0.3}
    adj[n] = res

In [16]:
def metapath_weight(path):
    #p = [u,t,t]
    path = path.split(' ')
    sum_of_w = 0
    pl = len(path)-1
    for i in range(pl):
        w = G[path[i]][path[i+1]][0]['weight']
        sum_of_w += w
    weight = sum_of_w/pl
    return weight

In [17]:
def int_float(v):
    if type(v) is float:
        v = int(v)
    return str(v)

In [18]:
def all_meta_paths(user,lmax,e):
    # O = set()
    P = set()
    M = {}
    Ptem = set()
    # wp_list = []
    for i in range(1,lmax+2):
        if i==1:
            # O.add(user)
            # print('o只包含user',O)
            for v,w in adj[user].items():
                P.add(' '.join([user,v]))
                # O.add(v)
        else:
            # print('当i=',i)
            Ptem = set()
            for pc in P.copy():
                pc_list = pc.split(' ')
                vc = pc_list[-1]
                mw = metapath_weight(pc)
                if G.nodes[vc]['type'] == 'thread' and mw >e:
                    # wp_list.append(mw)
                    # print('添加metapath:',pc)
                    M[pc] = mw
                    # M.append([pc,mw])
                if i != lmax+1:
                    C = set(adj[vc].keys())-set(pc_list)
                    if len(C) != 0:
                        for v in C:
                            new_pc = pc+' '+v
                            Ptem.add(new_pc)
                            # O.add(v)
                        # print('新增后临时路径数：',len(Ptem))
                P.remove(pc)
            P = Ptem
            # print('目前路径数：',len(P))
            # print('目前元路径数：',len(M))
    return M


In [19]:
import json
lmax = 3
e = 0.6
lalala = 0
file = open('data/metapath.txt','a+')
for user in user_char['portrait']:
    meta_path = {}
    M = all_meta_paths(user,lmax,e)
    meta_path[user] = M
    js = json.dumps(meta_path)
    lalala += 1
    if lalala%100 == 0:
        print(lalala)
    file.write(js)
    file.flush()
file.close()

100
200
300
400


In [20]:
import json
import pandas as pd

file = open('data/metapath.txt', 'r')
js = file.read()
js = js.replace("}{",", ")
dic = json.loads(js)
file.close()

In [21]:
def normalize(max,min,x):
    y = (x-min)/(max-min)
    return y

In [22]:
threads_meta = {}
for user,paths in dic.items():
    threads_temp = {}
    for p,w in paths.items():
        node_list = p.split()
        thread = node_list[-1]
        k = len(node_list)
        path_weight = w/(k-1)
        if thread not in threads_temp.keys():
            threads_temp[thread] = path_weight
        else:
            threads_temp[thread] += path_weight
    max_value = max(threads_temp.values())
    min_value = min(threads_temp.values())
    for k,v in threads_temp.items():
        threads_temp[k] = normalize(max_value,min_value,v)
    threads_meta[user] = threads_temp

In [24]:
thread_char = pd.read_csv('data/thread_character2000.csv',encoding='utf-8')
threads_time = {}
for user,thread in threads_meta.items():
    threads_temp= {}
    for t in thread.keys():
        w = thread_char['timestamp_new'][thread_char['thread_id']==int(t)].values[0]
        threads_temp[t] = w
    max_value = max(threads_temp.values())
    min_value = min(threads_temp.values())
    for k,v in threads_temp.items():
        threads_temp[k] = normalize(max_value,min_value,v)
    threads_time[user] = threads_temp

In [25]:
final_recommend = {}
time_tune = 0.5
for user,thread in threads_meta.items():
    threads_temp= {}
    for t,mw in thread.items():
        w = time_tune*threads_time[user][t]+(1-time_tune)*mw
        threads_temp[t] = w
    max_value = max(threads_temp.values())
    min_value = min(threads_temp.values())
    for k,v in threads_temp.items():
        threads_temp[k] = normalize(max_value,min_value,v)
    final_recommend[user] = sorted(threads_temp.items(), key = lambda kv:(kv[1], kv[0]),reverse=True)

In [26]:
file = open('data/result.txt','a+')
js = json.dumps(final_recommend)
file.write(js)
file.close()
